# Optical flow random image denoising

In [ ]:
local_debug = True

In [ ]:
import logging
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
#from ipywidgets import *
import cv2
import time
#import kernels
from skimage import io as skimage_io

In [ ]:
import logging
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
if local_debug:
    !ln -sf ../../information_theory/src/information_theory/ .
else:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
import information_theory  # pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
import denoising.image.OF_random as denoising

In [ ]:
GT = skimage_io.imread("http://www.hpca.ual.es/~vruiz/images/barb.png")

In [ ]:
plt.imshow(GT, cmap="gray")

In [ ]:
padded_GT = np.zeros(shape=(GT.shape[0]+32, GT.shape[1]+32), dtype=np.uint8)
padded_GT[16:-16, 16:-16] = GT
GT = padded_GT

In [ ]:
mean = 0
var = 1000
sigma = 20 #var**0.5
noise = np.random.normal(mean, sigma, GT.shape).reshape(GT.shape)
noisy = np.clip(a=GT.astype(np.float32) + noise, a_min=0, a_max=255).astype(np.uint8)

In [ ]:
plt.imshow(noisy, cmap="gray")

In [ ]:
logger.setLevel(logging.INFO)
denoiser = denoising.Monochrome_Denoiser(
    logger,
    pyramid_levels=3,
    window_side = 15,
    N_poly = 3,
    num_iterations = 3
)

In [ ]:
denoised_img, _ = denoiser.filter(noisy, GT, N_iters=20, RS_sigma=0.75)
#denoised_img, _ = denoiser.filter(noisy, GT, N_iters=80, RS_sigma=5.5)

In [ ]:
denoised = np.clip(denoised_img, a_min=0, a_max=255)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(16, 32))
axs[0].imshow(GT, cmap="gray")
axs[0].set_title("Original")
axs[1].imshow(noisy, cmap="gray")
axs[1].set_title(f"Noisy ({information_theory.distortion.SSIM(noisy, GT):3.2f})")
axs[2].imshow(denoised, cmap="gray")
axs[2].set_title(f"Denoised ({information_theory.distortion.SSIM(denoised, GT):3.2f})")
fig.tight_layout()
plt.show()

In [ ]:
np.mean(GT)

In [ ]:
np.mean(denoised)